Adapted from *Physics Simulations in Python -- A Lab Manual*, by Daniel V. Schroeder, Department of Physics, Weber State University, http://physics.weber.edu/schroeder/scicomp/

This work is licensed under the Creative Commons Attribution 4.0 International License.  To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/ or send a letter to Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

# Introduction

You now know enough about VPython to write your first simulation program.

The idea of a *simulation* is to program some system (such as a physical system following the laws of physics) into the
computer, and then let the computer calculate what happens as a function of
time, step by step into the future.  In this chapter, those our goal will be to predict the motion of
one or more objects subject to various forces.  Simulations let us do this for
*any* forces and *any* initial conditions, even when no explicit formula for the 
motion exists.

In this project you'll simulate the motion of a projectile, first in one
dimension and then in two dimensions.  When the motion is
purely vertical, the state of the
projectile is defined by its position, $y$, and its velocity,
$v_y$.  These quantities are related by
\begin{equation}
v_y = {dy\over dt} \approx {\Delta y\over \Delta t} 
  = {y_{\rm final} - y_{\rm initial} \over \Delta t}.
\end{equation}
In a computer simulation, we already know the current value of $y$ and want
to predict the future value.  So let's solve this equation for $y_{\rm final}$:
\begin{equation}
y_{\rm final} \approx y_{\rm initial} + v_y\,\Delta t.
\label{yFinal}
\end{equation}
Similarly, we can predict the future value of $v_y$ if we know the current
value as well as the acceleration:
\begin{equation}
v_{y,\rm final} \approx v_{y,\rm initial} + a_y\,\Delta t.
\label{vyFinal}
\end{equation}
These equations are valid for any moving object.  For a projectile moving
near earth's surface without air resistance, $a_y = -g$ (taking the $+y$
direction to be upward).  In general, $a_y$ is given by Newton's second law,
\begin{equation}
a_y = {\sum F_y\over m},
\label{NewtonsLaw}
\end{equation}
where $m$ is the object's mass and the various forces can depend on $y$,
$v_y$, or both.

In a computer simulation of one-dimensional motion, the idea is to start with
the state of the particle at $t=0$, then use \eqref{yFinal} through \eqref{NewtonsLaw} to calculate $y$ and $v_y$ at $t = \Delta t$, then repeat the
calculation for the next time interval, and the next, and so on.
Fortunately, computers don't mind doing repetitive calculations.

But there's one remaining issue to address before we try to program these
equations into a computer.  Equation \eqref{yFinal} is ambiguous regarding *which*
value of $v_y$ appears on the right-hand side.  Should we use the initial value,
or the final value, or some intermediate value?  In the limit $\Delta t \rightarrow 0$
it wouldn't matter, but for any nonzero value of $\Delta t$, some choices give
more accurate results than others.  The easiest choice is to use the
*initial* value of $v_y$, since we already know this value without any
further computation.  Similarly, the simplest choice in \eqref{vyFinal} is
to use the initial value of $a_y$ on the right-hand side.

With these choices, we can use the following Python code to simulate projectile motion
in one dimension without air resistance:
```
    while y > 0:
        ay = -g
        y += vy * dt    # use old vy to calculate new y
        vy += ay * dt   # use old ay to calculate new vy
        t += dt
```
This simple procedure is called the *Euler algorithm*, after the
mathematician Leonard Euler (pronounced "oiler"").  As we'll see, it is only one 
of many algorithms that give correct results in the limit $\Delta t\rightarrow 0$.

**Exercise.** In the cell below, write a program to simulate the motion of a dropped ball moving only in the vertical dimension, using the Euler algorithm as written in the code fragment above.  Represent the ball in the 3D graphics scene as a sphere with any reasonable radius (for display only---please ignore the radius in all physics calculations), and make a very shallow box at $y=0$ to represent the ground.  Use a light background color for eventual printing.  The ball should leave a trail of dots as it moves.  Be sure to put in the necessary code to initialize the variables (including **g**, putting all values in SI units), add a **rate** function inside the simulation loop, and update the ball's **pos** attribute during each loop iteration.  Also be sure to format your code to make it easy to read, with appropriate comments.  Use a time step (**dt**) of 0.1 second.  Start the ball at time zero with a height of 10 meters and a velocity of zero.  Notice that I've written the loop to terminate when the ball is no longer above $y=0$.  Test your program and make sure the animated motion looks reasonable.

In [37]:
from vpython import *

# Constants
g = 9.8  # Acceleration due to gravity (m/s^2)
dt = 0.1  # Time step 
t = 0  # Initial time
y = 10  # Initial height (m)
vy = 0  # Initial velocity 

# Create canvas
scene = canvas(title='Falling Ball Simulation',
               width=400,  # Reduced width for better focus
               height=300,
               center=vector(0, y/2, 0),  # Centering the view at half the initial height
               background=color.white)

# Create the ball
ball = sphere(pos=vector(0, y, 0), radius=0.2, color=color.blue, make_trail=True, trail_type='points', interval=10)

# Create the ground
ground = box(pos=vector(0, 0, 0), size=vector(10, 0.05, 10), color=color.green)

# Simulation loop
while y > 0:
    rate(10)
    ay = -g
    y += vy * dt    # use old vy to calculate new y
    vy += ay * dt   # use old ay to calculate new vy
    t += dt
    ball.pos = vector(0,y,0)



# Print the time of impact and the final velocity
print("Ball lands at t =", t, "seconds, with velocity", vy, "m/s")

<IPython.core.display.Javascript object>

Ball lands at t = 1.5000000000000002 seconds, with velocity -14.700000000000005 m/s


**Exercise.** Most of the space on your graphics canvas is wasted.  To fix this, in the previous code cell, set **scene.center** to half the ball's starting height (effectively pointing the "camera" at the middle of the trajectory), and set **scene.width** to 400 or less.

**Exercise** Let's focus our attention on the *time* when the ball hits the ground, and on the *final velocity* upon impact.  To see the numerical values of these quantities, add the following line to the end of your program:
```
  print("Ball lands at t =", t, "seconds, with velocity", vy, "m/s")
```
Here we're passing five successive parameters to the *print* function: three quoted strings that are always the same (called *literal* strings), and the two variables whose values we want to see.  If you look closely, you'll notice that the *print* function adds a space between successive items in the output.

In simulation codes it is crucial to validate not only the correctness of the code, but also its accuracy. After all, we take numbers, do some calculations, and return numbers. How do we these are correct? How accurate is the results. Sometimes, there are cases that we can calculate the expected result by hand. 

**Exercise.** Calculate the exact values of landing time and velocity, and write them in the markdown cell below. Compare these values with the results of your simulation.

**Answers:** 


To find the time it takes for an object to fa,y, we use the equation of motion:

$$ y = y_0 + v_{y0} \cdot t + \frac{1}{2} \cdot g \cdot t^2 $$

Where:
- \( y_0 = 10 \, \text{m} \) (initial height)
- \( v_{y0} = 0 \, \text{m/s} \) (initial velocity)
- \( g = 9.8 \, \text{m/s}^2 \) (acceleration due to gravity)

We set the final position \( y \) to 0 (ground level), and solve for \( t \):

$$ 0 = 10 + 0 \cdot t - \frac{1}{2} \cdot 9.8 \cdot t^2 $$
$$ 0 = 10 - 4.9 \cdot t^2 $$
$$ 4.9 \cdot t^2 = 10 $$
$$ t^2 = \frac{10}{4.9} $$
$$ t = \sqrt{\frac{10}{4.9}} $$
$$ t \approx \sqrt{2.04} $$
$$ t \approx 1.428 \, This gives us the time it takes for the ball to hit the ground.


t= 1.428 
**lly&=-\frac{1}{2}gt^{2}+10
\end{aligned}
$$
**

**Exercise.** The time and velocity printed by your program do not apply at the 
instant when the ball reaches $y=0$, because that instant occurs somewhere
in the middle of the final time interval.  Add some code after the end of the *while*
loop to estimate the time when the ball actually reaches
$y=0$.  (Hint:  Use the final values of **y** and **vy** to make your estimate.  The improved time 
estimate still won't be exact, but it will be much more accurate than what 
your program has been printing so far.)  Have the program print out the
improved value of $t$ instead, as well as an improved estimate of the velocity 
at this time.  Write your code in the cell below (not in the old cell).

In [14]:
from vpython import *

def simulate_falling_ball():
    # Constants
    g = 9.8  # Acceleration due to gravity 
    t = 0  # Initial time
    y = 10  # Initial height 
    vy = 0  # Initial vertical velocity (m/s)
    dt = 1e-5  # Time step

    # Create the canvas
    scene = canvas(title='Falling Ball Simulation',
                   width=400, height=300,
                   center=vector(0, y/2, 0),
                   background=color.white)
    ball = sphere(pos=vector(0, y, 0), radius=0.2, color=color.blue, make_trail=True, trail_type='points', interval=int(1 / dt))
    ground = box(pos=vector(0, 0, 0), size=vector(10, 0.05, 10), color=color.green)

    prev_y = y
    prev_t = t

    while y > 0:
        rate(1/dt)
        prev_y = y
        prev_t = t       
        ay = -g
        vy += ay * dt    # update velocity
        y += vy * dt     # update position
        ball.pos.y = y   # update ball's position in the environment
        t += dt          # increment time

    # when y<0 , we nee to calculate the v ata y=0
    if y < 0:
        t_land = prev_t + (0-prev_y)/(y-prev_y)*dt
        vy_land = vy + ay*(t_land-prev_t)
    else:              # if for some reason we got exactly y=0, there's no need for inderpolation
        t_land = t
        vy_land = vy

    # Print the results of the simulation
    print(f"Time of flight: {t_land:.6f} s, Final velocity: {vy_land:.6f} m/s")



# usage
simulate_falling_ball()  # Run the simulation



<IPython.core.display.Javascript object>

Time of flight: 1.428566 s, Final velocity: -14.000049 m/s


**Exercise.** The inaccuracy caused by the nonzero size of **dt** is called
*truncation error*.  You can reduce the size of the truncation error
by making **dt** smaller.  Try it!  
(You'll want to adjust the parameter of the *rate* function to make the program run faster when **dt** is small.  The slick way to do this is to make the parameter a *formula* that depends on **dt**. You'll also want to set the ball's **interval** attribute in a similar way, so the dot spacings are the same for any **dt**.)
How small must **dt** be to give results that
are accurate to four significant figures?  Justify your answer.

**Answers:** i did a loop for with differents "dt" variable, and as i see there is a start of convergent in the "time of flight" and in "time velocity" while we use at dt=1e-05


# Air resistance

There's not much point in writing a computer simulation when you
can calculate the exact answer so easily.  So let's make the problem more difficult
by adding some air resistance.  At normal speeds, the force of air resistance
is approximately proportional to the *square* of the projectile's velocity.  
This is because a faster projectile not only collides with *more* air
molecules per unit time, but also imparts more momentum to each molecule it hits.  So
we can write the magnitude of the air force as
\begin{equation}
|\vec F_{\rm air}| = c|\vec v|^2,  
\label{airForce}
\end{equation}
for some constant $c$ that will depend on the size and shape of the object
and the density of the air.
The *direction* of the air force is always directly opposite to the direction
of $\vec v$ (at least for a symmetrical, nonspinning projectile).

**Exercise.** Assuming that the motion is purely in the $y$ direction, write down
a formula for the $y$ component of the air force, in terms of $v_y$.  
Your formula should have the
correct sign for *both* possible signs of $v_y$.  (Hint:  Use an absolute value
function.) 

**Answer:** 

$$ F_{y,\text{air}} = -c \cdot v_y \cdot \left| v_y \right| $$ction.


**Excercise.** In the cell below, modify your program to include air resistance.
Define a new variable called **drag**, equal to the coefficient $c$ in \eqref{airForce} divided by the ball's mass.  Then add a term  for air resistance to the line that calculates **ay**.  Python's absolute
value function is called *abs()*.  Run the program
for the following values of **drag**:  0 (to check that you get the same
results as before), 0.01, 0.1, and 1.0.  Use the same initial conditions as
before, with a time step small enough to give about four significant figures.
Write down the results for the time of flight and final speed below.
There is a markdown cell below the code one for you to add explanation for your code.

In [41]:
# check the code litttl bit 

from vpython import *

def simulate_falling_ball(drag):
    # Constants
    g = 9.8  # Acceleration due to gravity (m/s^2)
    dt = 1e-5  # Time step for high precision
    y = 10  # Initial height (meters)
    vy = 0  # Initial vertical velocity (m/s)
    t = 0  # Initial time

    # Setup the 3D environment
    scene = canvas(title="Falling Ball Simulation with Drag", width=400, height=400, background=color.white)
    ground = box(pos=vector(0, -0.05, 0), size=vector(30, 0.1, 30), color=color.green)
    ball = sphere(pos=vector(0, y, 0), radius=0.5, color=color.purple, make_trail=True, trail_radius=0.05, interval=int(1/dt))

    # Setup graph for velocity vs time
    velocity_graph = graph(title=f"Velocity vs Time, Drag = {drag}", xtitle='Time (s)', ytitle='Velocity (m/s)', align='right')
    velocity_plot = gdots(graph=velocity_graph, color=color.blue, interval=10)

    # Simulation loop
    while y > 0:
        rate(1/dt)
        ay = -g - drag * abs(vy) * vy  # Calculate acceleration with air resistance
        vy += ay * dt  # Update velocity
        y += vy * dt  # Update position
        ball.pos.y = y  # Update ball's position
        velocity_plot.plot(t, vy)  # Plot current velocity
        t += dt  # Increment time

        # if y<0 , make linear interpolation 
        if y < 0:
            delta_t = -y / vy  # Time to backtrack to y = 0
            final_t = t + delta_t
            final_vy = vy + ay * delta_t
            print(f"Drag: {drag}, Time of flight: {final_t:.4f} s, Final velocity: {abs(final_vy):.4f} m/s")
            break


# loop for few values of drags 
drag_values = [0, 0.01, 0.1, 1]
for drag in drag_values:
    simulate_falling_ball(drag)


<IPython.core.display.Javascript object>

Drag: 0, Time of flight: 1.4286 s, Final velocity: 14.0000 m/s


<IPython.core.display.Javascript object>

Drag: 0.01, Time of flight: 1.4525 s, Final velocity: 13.3283 m/s


<IPython.core.display.Javascript object>

Drag: 0.1, Time of flight: 1.6743 s, Final velocity: 9.2053 m/s


<IPython.core.display.Javascript object>

Drag: 1, Time of flight: 3.4158 s, Final velocity: 3.1305 m/s


**Explanations**: Write your answer here! #explanation to you code -

1. Initializes the 3D canvas where the simulation will be visualized.

2. Places a sphere representing the ball and a box for the ground in the simulation.

after that we start the while loop which calculate the velocity and the time at hit the ground, Decreases due to gravity and drag, where drag is proportional to the square of velocity (drag * v * abs(v)).

$$ a_y = -g - \frac{c}{m} \left| v_y \right| v_y = -g - \text{drag} \left| v_y \right| v_y $$


meanwhile we Increases by v * dt, moving the ball based on its current velocity.
if y<0 we fix the issue to get more accurate values due to the simulation time step.
(Calculates delta_t, the time needed to backtrack to when the ball exactly reaches ground level (y = 0). Adjusts t and updates v to reflect this exact moment, ensuring physical accuracy.)



**Question:** What are the SI units of the **drag** constant in your program?

**Answer:** 1/m

**Question:** How can you tell that your program is accurate to about four significant
figures, when you no longer have an "exact" result to compare to?

**Answer:**
we can run our program with decreasing of dt values , when we start to seec convergence at four digits we can guess it's enough accurate .

**Exercise.** Modify your last code cell to plot a graph showing
the ball's velocity as a function of time.  (By default the graph will appear above or below the graphics canvas, and you may leave it there if you like.  If you would prefer to place the graph to the right of the canvas, you can do so by creating the graph first, setting its attribute **align="right"**, and immediately plotting at least one point on the graph to make it appear before you set up the canvas.)  Use the **xtitle** and **ytitle** attributes to label both axes of the graph appropriately, including the units of the plotted quantities.  Use the **interval** parameter of the **gdots** function to avoid plotting a dot for every loop iteration (which would be pretty slow).  Run your program again with **drag** equal to 1.0.  Discuss the results briefly.

**Discussion:** 1.effct higt drag : with a high drag coeffiecient we will get a rapid decrese in the velocity as a funtion of time, faster than a case with a low coeffienct drag . 
2. velocity curve :as the coeffiecnt drag is higher we will get a steep decline in the velocity cause the resistence of the air, and that also addtect the time it take for a bullet to reach to a ground .

**Exercise.** When the projectile is no longer accelerating, the forces acting
on it must be in balance.  Use this fact to calculate your projectile's terminal
speed by hand, and compare to the result of your computer simulation.

**Answers:** Write your answer here!

**Answers:** Using formula 
$$
a_{\text{land}}=-g+\text{drag}\left|v_{\text{land}}\right|^{2}=0\iff\left|v_{\text{land}}\right|^{2}=\frac{g}{\text{drag}}\iff v_{\text{land}}=\sqrt{\frac{g}{\text{drag}}}
$$
and foe $\text{drag}=1$ we will get $v_{\text{land}}=\sqrt{9.8} m/s^{2} \approx3.1305 m/s^{2}$.

# A better algorithm

The Euler algorithm used above is sufficiently inaccurate that you've needed to use pretty small
values of **dt**, making the calculation rather lengthy.  Fortunately, it isn't
hard to improve the Euler algorithm.

The crux of the algorithm is explained in the figure below: the derivative of a function at the
middle of an interval (point $B$) is a much better approximation to the average
slope ($AC$) than the derivative at the beginning of the interval (point $A$).

![dummy](http://www.math.tau.ac.il/~haimav/SlopeAtMiddle.png)

The Euler algorithm uses the values of $v_y$ and $a_y$ at the
*beginning* of the time interval to estimate the changes in the
position and velocity, respectively.  A much better approximation would be to
instead use the values of $v_y$ and $a_y$ at the *middle* of the time
interval.  Unfortunately, these values are not yet known. 
But even a rough estimate of these values should be better than none at all.  Here is
an improved algorithm that uses such a rough estimate:

1. Use the values of $v_y$ and $a_y$ at the beginning of the interval to estimate the position and velocity at the middle of the time interval.

2. Use the estimated position and velocity at the middle of the interval to calculate an estimated acceleration at the middle of the interval.

3. Use the estimated $v_y$ and $a_y$ at the middle of the interval to calculate the changes in $y$ and $v_y$ over the whole interval.

This procedure is called the *Euler-Richardson algorithm*, also known
as the *second-order Runge-Kutta algorithm*. 

Here is an implementation
of the Euler-Richardson algorithm in Python for a projectile moving in one dimension,
without air resistance:
```
    while y > 0:
        ay = -g                      # ay at beginning of interval
        ymid = y + vy*0.5*dt         # y at middle of interval
        vymid = vy + ay*0.5*dt       # vy at middle of interval
        aymid = -g                   # ay at middle of interval
        y += vymid * dt
        vy += aymid * dt
        t += dt
```

The acceleration calculations in this example aren't very interesting, because
$a_y$ doesn't depend on $y$ or $v_y$.  Still, the basic idea is to estimate
$y$, $v_y$, and $a_y$ in the middle of the interval and then use these values
to update $y$ and $v_y$.  Although each step of the Euler-Richardson algorithm
requires  roughly twice as much calculation as the original Euler algorithm, it is
usually many times more accurate and therefore allows us to use a much larger time
interval.

**Exercise.** Write down the correct modifications to the lines that calculate
**ay** and **aymid**, for a projectile falling *with* air resistance.
(Be careful to use the correct velocity value when calculating **aymid**!)

**Answer:**
```
    while y > 0:
        ay = -g - c * vy**2          # ay at beginning of interval
        ymid = y + vy*0.5*dt         # y at middle of interval
        vymid = vy + ay*0.5*dt       # vy at middle of interval
        aymid = -g - c * vymid**2    # ay at middle of interval
        y += vymid * dt
        vy += aymid * dt
        t += dt
```






**Questions.** One of the lines in the Euler-Richardson implementation above is not
needed, even when there's air resistance.  Which line is it, and why do you think
we included it if it isn't needed?

**Answer:**    
```
ymid = y + vy * 0.5 * dt            # y at middle of interval
```
the reason :
this intermediate position ymid is not used for any other calculations. The updated position and velocity are calculated directly using the velocities at the vymid and aymid.

**Exercise.** In the cell below, rewrite your program to use the Euler-Richardson
algorithm.  For a **drag** constant of 0.1 and
the same initial conditions as before ($y=10m$, $v_y = 0$),
how small must you now make **dt** to get answers accurate to four significant
figures?  Justify your answer.  (You should find that **dt** can now be *significantly* larger
than before.  If this isn't what you find, there's probably an error in your
implementation of the Euler-Richardson algorithm.)

In [28]:
from vpython import sphere, vector, color, box, canvas, rate

def simulate_falling_ball_with_euler_richardson(drag):
    g = 9.8  # Gravity (m/s^2)
    y = 10  # Initial height (m)
    vy = 0  # Initial velocity (m/s)
    t = 0  # Initial time
    dt = 1e-4  # steps time
    # Setup the 3D environment
    scene = canvas(title="Falling Ball Simulation", width=400, height=400, background=color.white)
    ground = box(pos=vector(0, -0.05, 0), size=vector(30, 0.1, 30), color=color.green)
    ball = sphere(pos=vector(0, y, 0), radius=0.5, color=color.purple, make_trail=True)

    
    # Simulation loop
    while y > 0:
        rate(1/dt)  # Adjusted rate for better performance
        ay = -g - drag * vy * abs(vy)  # Initial acceleration
        vy_mid = vy + ay * 0.5 * dt  # Midpoint velocity estimate
        y_mid = y + vy_mid * 0.5 * dt  # Midpoint position estimate
        ay_mid = -g - drag * vy_mid * abs(vy_mid)  # Midpoint acceleration

        vy += ay_mid * dt  # Update velocity using midpoint acceleration
        y += vy_mid * dt  # Update position using midpoint velocity
        ball.pos.y = y  # Update the visual position
        t += dt  # Increment time

        if y < 0:  # Check if below ground to interpolate
            delta_t = -y / vy_mid  # Time to backtrack to y=0
            t += delta_t
            vy += ay_mid * delta_t
            y = 0

    print(f"Drag constant: {drag:.2f}, Time of flight: {t:.6f} s, Final velocity: {vy:.6f} m/s")

# simulation while deag =0.1
simulate_falling_ball_with_euler_richardson(drag=0.1)


<IPython.core.display.Javascript object>

Drag constant: 0.10, Time of flight: 1.674282 s, Final velocity: -9.205278 m/s


**Answers:** we can see which with $dt=10^{-4}$  we get an answers accurate to four significant figures. the justification is by convergence of "time of flight" and "final velocity".

# Two-dimensional projectile motion

Simulating projectile motion is only slightly more difficult in two dimensions
than in one.  To do so you'll need an $x$ variable for every $y$ variable, and
about twice as many lines of code to initialize these variables and update
them within the simulation loop.  To calculate the $x$ and $y$ components of the
drag force, it's helpful to draw a picture:

![dummy](http://www.math.tau.ac.il/~haimav/AirForce.png)

Assuming that the drag force is always
opposite to the velocity vector, the similar triangles in this diagram can
be used to express the force components in terms of the velocity components.

**Exercise.** Derive formulas 
for the $x$ and $y$ components of
the drag force, written in terms of $v_x$ and $v_y$.  The magnitude of the
drag force is again given by \eqref{airForce}. 

**Answer:** 
The x-component of the drag force is given by:
$$ f_{\text{drag}, x} = -c \cdot v_x \cdot \sqrt{v_x^2 + v_y^2} $$

The y-component of the drag force is given by:
$$ f_{\text{drag}, y} = -c \cdot v_y \cdot \sqrt{v_x^2 + v_y^2} $$



**Excercise.** In the cell below, write the Python code for calculating the acceleration components, **ax** and **ay**, for a projectile moving in two dimensions with air resistance.  The Python function for taking a square root is *sqrt()*.  To square a quantity you can either just multiply it by itself, or use the Python exponentiation operator, \*\*. *There is no need to actually run the cell, just write the code.*

In [30]:
while False: #use false to enable the code run witout error 
    v = sqrt(vx**2 + vy**2)
    ax = - drag * vx * v        
    ay = -g - drag * vy * v

**Exercise.** Now think about implementing the Euler-Richardson algorithm in two dimensions.  For each line of the code we gave for the Euler-Richardson algorithm that calculates a $y$ variable, you'll need to add a line to calculate the corresponding $x$ variable.  But the order of these lines matters!  Should you alternate $x,y,x,y\ldots\,$, or should you calculate all the $x$ variables first and then all the $y$ variables, or vice-versa?  Explain carefully.

**Answer:** 
from mathematical approach the result of the two type of calculation suppose to be same, the components are independent.
.but i think to use at sequential calculations of preferred , first calculate all the x variable and after that calculate all y variable (or vice versa).)
.

two resoans :
1. Consistency: This method ensures that all updates rely on the same baseline data, which helps prevent the introduction of errors that can occur when some variables are updated mid calculation using values that no longer reflect the starting conditions of the current timestep

2. 
Symmetry and Stability:Consistent at mid-point values across all dimensions (x and y) ensure the stability and accuracy enhancements the method is de.helly 
**

**Exercise.** In the cell below, write code to simulate projectile motion in two dimensions, for the specific case of a ball launched from the origin at a given initial speed and angle that are set near the top of the program.  Again use a sphere to represent the ball, leaving a trail as it moves, and use a very shallow box to represent the ground.  Allow for the ball to travel as far as about 150 meters in the $x$ direction before it lands, sizing the box and setting **scene.center** appropriately. Use the Euler-Richardson algorithm, with a time step of 0.01 second.  Run your program and check that everything seems to be working.

In [45]:
from vpython import sphere, vector, color, box, canvas, rate, sqrt, sin, cos, radians

def simulate_projectile_motion(v0, theta, dt=0.01, c=0.01, g=9.8):
 
    # Convert angle to radians
    theta_radians = radians(theta)

    # Initial conditions
    x, y = 0, 0  # our initial point
    vx = v0 * cos(theta_radians)  # velocity on x axe
    vy = v0 * sin(theta_radians)  # velocity on y axe
    t = 0  # Time starts at zero
    ymax = 0  # variable to calculate y max 

    # Create canvas
    scene = canvas(title='Projectile Motion Simulation',
                   width=1200, height=600,
                   center=vector(75, 25, 0),
                   background=color.white)
    ball = sphere(pos=vector(x, y, 0), radius=1, color=color.blue, make_trail=True)
    ground = box(pos=vector(75, 0, 0), size=vector(300, 2, 10), color=color.green)

    # Simulation loop
    while ball.pos.y >= 0:
        rate(150)  # Control the rate of the simulation to make it visually manageable

        if y > ymax:
            ymax = y  # Update maximum height

        # Compute accelerations
        ax = -c * vx * sqrt(vx**2 + vy**2)
        ay = -g - c * vy * sqrt(vx**2 + vy**2)

        # Midpoint calculations
        x_mid = x + vx * 0.5 * dt
        y_mid = y + vy * 0.5 * dt
        vx_mid = vx + ax * 0.5 * dt
        vy_mid = vy + ay * 0.5 * dt
        ax_mid = -c * vx_mid * sqrt(vx_mid**2 + vy_mid**2)
        ay_mid = -g - c * vy_mid * sqrt(vy_mid**2 + vy_mid**2)

        # Final updates
        x += vx_mid * dt
        y += vy_mid * dt
        vx += ax_mid * dt
        vy += ay_mid * dt
        t += dt

        # Update position ball
        ball.pos = vector(x, y, 0)

        # Check for landing
        # interpollation 
        if y < 0:
            delta_t = -y / vy
            final_t = t + delta_t
            final_x = x + vx * delta_t
            break

    # Output results , printing 
    print("Simulation complete.")
    print(f"drag: {c:.3f} m")
    print(f"Initial speed: {v0:.3f} m/s, Launch angle: {theta} degrees, Drag constant: {c}")
    print(f"Landing time: {final_t:.3f} s")
    print(f"Range of the projectile: {final_x:.3f} m")
    print(f"Maximum height: {ymax:.2f} m")

# Example
simulate_projectile_motion(v0=30, theta=45)

# done and good 

<IPython.core.display.Javascript object>

Simulation complete.
drag: 0.010 m
Initial speed: 30.000 m/s, Launch angle: 45 degrees, Drag constant: 0.01
Landing time: 3.802 s
Range of the projectile: 56.603 m
Maximum height: 17.689 m


**Excercise.** Add code to the last cell to calculate and display the landing time, the value of $x$ at this time (that is, the *range* of the projectile), and the projectile's maximum height.  Use interpolation for the first two quantities, as you did in the one dimensional case.  For the maximum height, you'll need to test during each iteration whether the current height is more than the previous maximum.  To do this you can use an *if* statement, whose syntax is similar to that of a *while* loop:
```
    if y > ymax:
        ymax = y
```
Use *print* functions to display all three of your calculated results, along with the initial speed and angle, and the **drag** constant.

**Excercise.** Check that your program gives the expected results when there is no air resistance, for a launch angle of $45^\circ$ and your choice of launch speed.  Write down your results, along with your calculations of the expected results.

### **Answer:** 

## Theoretical Calculations for Projectile Motion

To verify the accuracy of the program without air resistancewe usean calculate thfunction :s.

### Key Formlas

For projectile motion without air resistance:

1. **Range (R)**:
   $$ R = \frac{v_0^2 \sin(2\theta)}{g} $$
   where \( v_0 \) is the initial speed, \( \theta \) is the launch angle in degrees, and \( g \) is the acceleration due to gravity.

2. **Time of Flight (T)**:
   $$ T = \frac{2 v_0 \sin(\theta)}{g} $$

3. **Maximum Height (H)**:
   $$ H = \frac{v_0^2 \sin
^### Case Variables

In our simulation, we use the following parameters:
- Initial velocity (`v0`): 30 m/s
- Launch angle (`theta`): 45 degrees
- Acceleration due to gravity (`g`): 9.8 m/s²

### Simulation Output

When we execute the Python code for calculating the projectile motion, we get the next results:

- **Calculated Range:** 91.8367 m
- **Calculated Time of Flight:** 4.3292 s
- **Calculated Maximum Height:** 22.9592 m2 m



  g = 9.8 \) m/s²:

### Python
ht, and maximum height:

```python
from math import sin, cos, radians, pi

# Constants
v0 = 30  # m/s
theta_degrees = 45
theta_radians = radians(theta_degrees)
g = 9.8  # m/s^2

# Calculations
Range = (v0**2 * sin(2 * theta_radians)) / g
Time_of_Flight = (2 * v0 * sin(theta_radians)) / g
Max_Height = (v0**2 * (sin(theta_radians))**2) / (2 * g)

# Print results
print(f"Calculated Range: {Range:.4f} m")
print(f"Calculated Time of Flight: {Time_of_Flight:.4f} s")
print(f"Calculated Maximum Height: {Max_Height:.4f} m")



**Exercise.** To control the number of displayed digits, you can use the following (admittedly arcane) syntax:
```
    Maximum height = "{:.2f}".format(ymax)
```
Here we're creating a string object (in quotes) and then calling its associated *format* function with the parameter **ymax**.  This function replaces the curly braces, and what's between them, with the value of **ymax**, rounded to two decimal places.  (To change this to three decimal places, you would just change *.2f* to *.3f*; the *f* stands for *floating-point* format.)  Make the needed changes to display all three of the calculated results to just four decimal places.

# A graphical user interface

Are you tired of having to edit your code and rerun your programs every time you want to change one of the constants or initial conditions?  A more convenient approach---at least if you'll be running a simulation more than a handful of times---is to create a *graphical user interface*, or *GUI*, that lets you adjust these numbers and repeat the simulation while the program is running.

Creating a graphical user interface using
VPython is easy.  The documentation refers
to its GUI controls as *widgets*, and in this project we'll use three of them:  the button, the slider, and the dynamic text widget.

Here's some minimal code to create a button that merely displays a message:
```
    def launch():
        print("Launching the projectile!")

    button(text="Launch!", bind=launch)
```
The first two lines define a *new function* called *launch*, and the last line creates a button that is *bound* to this function, so the function is called whenever the button is pressed.

**Exercise.** Copy the code from the last code cell to the cell below, and add the code above for adding the button at the end. Try it. Does the button continue to work after the evaluation of the cell has completed (indicated by the fact that the cell is given a number in the brackets)?

In [50]:
from vpython import *

def launch():
    # Clear data
    scene.clear()
    
    # Re-setup the scene
    setup_scene()

    print("Launching the projectile!")
    
    # Start the simulation
    simulate_projectile_motion(v0=100, theta=45)




def setup_scene():
    global scene, ball, ground
    scene = canvas(title='Projectile Motion Simulation',
                   width=1200, height=600,
                   center=vector(75, 25, 0),
                   background=color.white)
    ball = sphere(radius=1, color=color.blue, make_trail=True)
    ground = box(pos=vector(75, 0, 0), size=vector(300, 2, 10), color=color.green)



def simulate_projectile_motion(v0, theta, dt=0.01, c=0.01, g=9.81):
    theta_radians = radians(theta)
    x, y = 0, 0
    vx = v0 * cos(theta_radians)
    vy = v0 * sin(theta_radians)
    t = 0
    ymax = 0
    
    while ball.pos.y >= 0:
        rate(100)
        if y > ymax:
            ymax = y
        
        ax = -c * vx * sqrt(vx**2 + vy**2)
        ay = -g - c * vy * sqrt(vx**2 + vy**2)
        x_mid = x + vx * 0.5 * dt
        y_mid = y + vy * 0.5 * dt
        vx_mid = vx + ax * 0.5 * dt
        vy_mid = vy + ay * 0.5 * dt
        ax_mid = -c * vx_mid * sqrt(vx_mid**2 + vy_mid**2)
        ay_mid = -g - c * vy_mid * sqrt(vy_mid**2 + vy_mid**2)
        
        x += vx_mid * dt
        y += vy_mid * dt
        vx += ax_mid * dt
        vy += ay_mid * dt
        t += dt
        
        ball.pos = vector(x, y, 0)
        if y < 0:
            delta_t = -y / vy
            final_t = t + delta_t
            final_x = x + vx * delta_t
            break

    print("Simulation complete.")
    print(f"Initial speed: {v0} m/s, Launch angle: {degrees(theta_radians)} degrees, Drag constant: {c}")
    print(f"Landing time: {final_t:.4f} s")
    print(f"Range of the projectile: {final_x:.4f} m")
    print(f"Maximum height: {ymax:.4f} m")

# Setup initial scene
setup_scene()

# Add a button to launch the simulation
button(text="Launch!", bind=launch)



<IPython.core.display.Javascript object>

You might wonder how to control *where* the button appears.  VPython doesn't give you nearly as much control over the placement as would an environment for developing commercial software.  By default, new widgets are placed immediately below the **scene** canvas, in what's called its *caption*.  There are a few other placement options that you can read about on the Widgets documentation page if you like.

More importantly, your Launch! button doesn't yet do what we want, namely launch the projectile!  One way would be to move all your initialization and simulation code into the definition of the new *launch* function. We will take a more difficult path, but one that will use a program structure that will be more useful in future projects. 

The basic idea is to turn your *while* loop into an *infinite* loop that runs forever, but to execute most of the code inside the loop only if the simulation is supposed to be "running".  The following code provides a basic outline.
```
    while True:
        rate(100)
        if running:
            # carry out an Euler-Richardson step
            if y < 0:
                running = False
                # print out results
``` 
The new idea here is the introduction of a *boolean* variable called **running*, whose value is always one of the two boolean constants **True** or **False** (note that these are capitalized in Python).  When **running** is **False**, we merely call the *rate* function to delay a bit before the next loop iteration.  When **running** is **True**, we carry out a time-integration step (using the code you've already written, omitted here for brevity), then test whether the ball has dropped below ground level, in which case we set ```running = False``` and print out the results.

**Exercise.** Insert this new code into a new cell, replacing the two comments with the code you've already written to carry out the time step and print the results.  Also add a line near the top of the program to initially set ```running = True```.  Test the program to verify that it works exactly as before.
**Important:** When you run the code, because of the infinite loop, the cell continues to run and it's evaluation never stops. This is indicated by the * in the brackets near the cell. This will prevent you from running this cell or any other cell. To stop the execution, use the menu item Kernel->Interrupt.

In [ ]:
from vpython import *


# build the simulation
# Setup the scene globally
scene = canvas(title='Projectile Motion Simulation',
               width=1200, height=600,
               center=vector(75, 25, 0),
               background=color.white)
ball = sphere(radius=1, color=color.blue, make_trail=True)
ground = box(pos=vector(75, 0, 0), size=vector(300, 2, 10), color=color.green)

# Constants
dt = 0.01  # Time step
g = 9.8  # Acceleration due to gravity (m/s^2)
running = False  # Initially, the simulation is not running

def adjustAngle():
    angleReadout.text = f"{angleSlider.value} degrees"

def adjustSpeed():
    speedReadout.text = f"{speedSlider.value} m/s"

def adjustDrag():
    dragReadout.text = f"{dragSlider.value:.3f}"

# UI for angle control
scene.append_to_caption("\n\nAngle (degrees): ")
angleSlider = slider(min=0, max=90, value=45, step=1, bind=adjustAngle)
angleReadout = wtext(text=f"{angleSlider.value} degrees")
scene.append_to_caption("\n")  # New line for clarity

# UI for speed control
scene.append_to_caption("\nSpeed (m/s): ")
speedSlider = slider(min=0, max=50, value=20, step=1, bind=adjustSpeed)
speedReadout = wtext(text=f"{speedSlider.value} m/s")
scene.append_to_caption("\n")  # New line for clarity

# UI for drag control
scene.append_to_caption("\nDrag Coefficient: ")
dragSlider = slider(min=0, max=1.0, value=0.01, step=0.005, bind=adjustDrag)
dragReadout = wtext(text=f"{dragSlider.value:.3f}")
scene.append_to_caption("\n")  # New line for clarity


def exit_simulation():
    global running
    running = False
    raise SystemExit("Simulation exited by user.")  # Attempt to terminate the notebook cell execution


def stop_simulation():
    global running
    running = False

def clear_trails():
    global x, y
    x, y = 0, 0  # Reset position to the origin
    ball.pos = vector(x, y, 0)  # Set the ball's position to the origin
    ball.clear_trail()  # Clear the trail

def clear_and_relaunch():
    clear_trails()
    launch()  # Immediately re-launch the simulation



# Function to launch the simulation
def launch():
    global x, y, vx, vy, running
    # Retrieve values from sliders
    v0 = speedSlider.value
    theta = angleSlider.value
    drag = dragSlider.value
    theta_radians = radians(theta)
    # Set initial conditions based on slider values
    x, y = 0, 0
    vx = v0 * cos(theta_radians)
    vy = v0 * sin(theta_radians)
    ball.pos = vector(x, y, 0)
    ball.clear_trail()
    running = True


# Buttons
button(text="Launch!", bind=launch)
button(text="Stop", bind=stop_simulation)
button(text="Clear Trails", bind=clear_trails)
button(text="Exit", bind=exit_simulation)  # Bind the exit function
button(text="Clear & Relaunch", bind=clear_and_relaunch)


def run_simulation():
    global x, y, vx, vy, running
    while True:
        rate(100)
        if running:
            # Perform simulation steps
            ax = -dragSlider.value * vx * sqrt(vx**2 + vy**2)
            ay = -g - dragSlider.value * vy * sqrt(vx**2 + vy**2)
            vx += ax * dt
            vy += ay * dt
            x += vx * dt
            y += vy * dt
            ball.pos = vector(x, y, 0)
            if y < 0:
                running = False
                print(f"Simulation complete. Landing position: x = {x:.4f} m, y = {y:.4f} m")
                print(f"Final velocity: vx = {vx:.4f} m/s, vy = {vy:.4f} m/s")

# excute 
run_simulation() 

<IPython.core.display.Javascript object>

**Exercise.** You're now ready to activate your Launch! button.  To do so, insert the following two lines into the indented body of the *launch* function definition:
```
    global running
    running = True
```
Also change your initialization line near the top of the program to set ```running = False``` instead of **True**.  Test your program again and verify that the projectile is not launched until you press the button. Remember that to exit cell evaluation you use Kernel->Interrupt.

**Exercise.** To allow multiple launches, simply move all of the relevant code to initialize the variables into your *launch* function.  You'll also need to add their names, separated by commas, to the *global* statement.  Leave the initializations of **dt** and **g** outside the function definition, since those values never change.  Check that you can now launch the projectile repeatedly.

**Exercise.** Using the menu item Kernel->Interrupt can be tedious and unintuitive. Instead, add another button called Exit that will cause the cell evaluation to terminate. 

Before going on, let us pause to explain that *global* statement.  In Python, any variable that you change inside a function definition is *local* by default, meaning that it exists only within the function definition and not outside it.  In longer programs, this behavior is a very good thing, because it frees you, when you're writing a function definition, from having to worry about which variable names have already been used elsewhere in the program.  But this means that when you *do* want to change a *global* variable (that is, one that belongs to the larger program), you need to "declare" it as *global* inside your function.   Notice that you need to use *global* only if your function is *changing* the variable; you don't need it just to "read" the value of a variable.  And for technical reasons, this means that you don't need to declare graphics objects as global variables just to change their attributes.

Now let's add some controls to let you vary the launch conditions.  The best way to adjust a numerical parameter is usually with a slider control.  Here is how you can add one to adjust the launch angle:
```
    def adjustAngle():
        pass    # function does nothing for now
    scene.append_to_caption("\n\n")
    angleSlider = slider(left=10, min=0, max=90, step=1, value=45, 
                            bind=adjustAngle)
```
The *slider* function creates the slider, and most of its parameters indicate which numerical values to associate with the allowed slider positions.  The **value** parameter is set to an initial value (here intended to be 45 degrees), but will change when you actually adjust the slider.  The **left** parameter and the *append_to_caption* function on the previous line merely insert some space around the slider in the window layout ("\\n" is the code for "new line").  The **bind** parameter, as before, binds a function to this control; that function (*adjustAngle*) will be called whenever you adjust the slider.  For now I've used Python's *pass* statement to make the function do nothing.

**Exercise.** Put this code into your program, and check that the slider appears underneath the Launch button.  Then modify the code that sets the ball's initial launch angle so it uses **angleSlider.value** instead of whatever angle you were giving it before.  You should now be able to launch multiple projectiles at varying angles.  Try it!

**Exercise.** The only problem now is that you can't tell exactly what angle the slider is set to -- at least not until you actually click Launch! and see the output of your *print* function.  To give the slider a numerical display, add the following code right after the line that creates the slider:
```
    scene.append_to_caption("    Angle = ")
    angleSliderReadout = wtext(text="45 degrees")
```
Then replace the *pass* statement in the *adjustAngle* function with:
```
    angleSliderReadout.text = str(angleSlider.value) + " degrees"
```
(This line uses the + operator to join two strings together: an example of what's called *operator overloading*.)  Verify that the slider's numerical readout now works as it should.

**Exercise.** Add two more sliders, with numerical readouts, to control the ball's initial speed and the **drag** constant.  Let the launch speed range from 0 to 50 m/s in increments of 1 m/s, and let the **drag** constant range from 0 to 1.0 in increments of 0.005 (in SI units).  Test these sliders to make sure they work as they should.

**Exercise.** Add a second button (on the same caption line as the Launch! button) to clear away the trails from previous launches and start over.  To do this, the bound function should set the ball's position back to the origin and then call the ball's *clear_trail* function (with no parameters). 

**Exercise.** Run your  program, experimenting with various values
of the drag coefficient, launch speed, and launch angle.  You may recall from introductory physics that when there is no air resistance, the maximum range for a given launch speed occurs at an angle of $45^\circ$.  Check this for a launch speed of 25 m/s, then increase the drag coefficient to 0.1 and find the angle that gives the maximum range.  What angle did you get? Explain why you would expect the optimum angle to be less when there is air resistance.  

**Answer:** 

for our condition of drag =0.1 and luanch speed =25 , i check many variables of theta\angle and found whihch for theta =33 , we get the maximum range of x .


Simulation complete. Landing position: x = 14.5028 m, y = -0.0004 m
Final velocity: vx = 3.8505 m/s, vy = -6.8792 m/s



**Exercise.** The maximum speed of a batted baseball is about 110 mph, or about 50 m/s.
At this speed, the ball's drag coefficient (as defined in your program) is
approximately $0.005~{\rm m}^{-1}$.  Using your program with these inputs, estimate
the maximum range of a batted baseball, and the angle at which the maximum range
is attained.  Write down and justify your results.
Is your answer reasonable, compared to the typical distance of an
outfield fence from home plate (about 350--400 feet)?  For the same initial speed
and angle, how far would the baseball go without air resistance?

**Answer:** 
with air resistnece we will get the maximum range when the angle will be 40 , i placed values and check when we get the maximum and the "Range of the projectile: 137.7650 m.

the values from the program :
Simulation complete. Landing position: x = 137.7650 m, y = -0.0021 m
s
without air reistence the Range of the projectile: 251.22436 m, almot 2 times bigger . 


in the cells below there are the program which i used to calculate this .
lly

In [3]:
from vpython import sphere, vector, color, box, canvas, rate, radians, cos, sin, sqrt

# Setup the scene globally
scene = canvas(title='Projectile Motion Simulation',
               width=1200, height=600,
               center=vector(75, 25, 0),
               background=color.white)
ball = sphere(radius=1, color=color.blue, make_trail=True)
ground = box(pos=vector(75, 0, 0), size=vector(300, 2, 10), color=color.green)

# Constants
dt = 0.0001  # Time step
g = 9.8  # Acceleration due to gravity (m/s^2)
running = True  # Start simulation immediately

# Initial conditions for simulation
v0 = 50  # Initial speed in m/s
theta = 40  # Launch angle in degrees
drag = 0.005  # Drag coefficient

# Convert angle to radians for calculation
theta_radians = radians(theta)

# Calculate initial velocity components
vx = v0 * cos(theta_radians)
vy = v0 * sin(theta_radians)

x, y = 0, 0  # Initial position of the projectile
ball.pos = vector(x, y, 0)  # Set initial position of the ball

def run_simulation():
    global x, y, vx, vy, running
    while running:
        rate(1/dt)
        # Calculate acceleration components
        ax = -drag * vx * sqrt(vx**2 + vy**2)
        ay = -g - drag * vy * sqrt(vx**2 + vy**2)

        # Update velocities
        vx += ax * dt
        vy += ay * dt

        # Update positions
        x += vx * dt
        y += vy * dt
        ball.pos = vector(x, y, 0)

        # Check if the projectile has hit the ground
        if y < 0:
            running = False
            print(f"Simulation complete. Landing position: x = {x:.4f} m, y = {y:.4f} m")
            print(f"Final velocity: vx = {vx:.4f} m/s, vy = {vy:.4f} m/s")

# Start the simulation
run_simulation()


<IPython.core.display.Javascript object>

Simulation complete. Landing position: x = 137.7650 m, y = -0.0021 m
Final velocity: vx = 17.1181 m/s, vy = -23.7047 m/s


Congratulations---you're now finished with this project!  Please turn in the filled notebook file.